# Retrieaval evaluation
In this notebook we evaluate different retrieval methods.
For perform it we need:
* create ground truth dataset, genereate question for our answers
* get search_results for our questions
* measure performanse of search methods

We will try:
* minisearch.py
* elasticsearch (text search)
* elasticsearch (vector search)

And separately evaluate search with chunking (split by 1000 symbols)

In [ ]:
To generate ground-truth dataset we need 